In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
import feature_manager as fma
import classifier.multi_dnn_classifier as dnn
from random import randint
from keras import callbacks, losses
import visualizer
from keras.utils import np_utils
from sklearn.metrics import classification_report, confusion_matrix
import importlib
import utils

2023-01-21 02:11:20.516305: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-21 02:11:20.516410: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-21 02:11:20.516424: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-21 02:11:21.580415: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-21 02:11:21.580462: I tensorflow/compiler/xla/stream_exe

In [8]:
importlib.reload(fma)

<module 'feature_manager' from '/home/andy/CryptoTradingPlatform/TraderRobot/project/feature_manager.py'>

In [9]:
symbol = "BTCUSDT"
trade_tf = "1d"
granular_tf = "1m"

tp = 0.1
sl = 0.08
md = 12

fm = fma.FeatureManager(
    target_col="trade_signal"
)

fm.import_data(symbol=symbol,trade_timeframe=trade_tf,granular_timeframe=granular_tf)

fm.prepare_trade_forward_data(
    take_profit_rate=tp,
    stop_loss_rate=sl,
    max_duration=md,
)

Imported ../data/BTCUSDT-1d.csv with 1963 rows
Imported ../nocommit/BTCUSDT-1m.csv with 2817999 rows
Scanning 12 future timeframes to build trade signal: 
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
Label producing completed. 
 Value counts:
1    695
2    638
0    630
Name: trade_signal, dtype: int64


In [10]:
fm.build_features(lags=90)

Calculating features...

Adding feature: returns, dir, sma, boll, boll7, boll14, boll21, min, min7, min14, min21, max, max7, max14, max21, mom, mom7, mom14, mom21, vol, vol7, vol14, vol21, obv, mfi7, mfi14, mfi21, rsi7, rsi14, rsi21, adx7, adx14, adx21, roc, roc7, roc14, roc21, atr7, atr14, atr21, bop, ad, adosc, trange, ado, willr7, willr14, willr21, dx7, dx14, dx21, trix, ultosc, high, low, 

Normalizing feature: returns_lag_1, returns_lag_2, returns_lag_3, returns_lag_4, returns_lag_5, returns_lag_6, returns_lag_7, returns_lag_8, returns_lag_9, returns_lag_10, returns_lag_11, returns_lag_12, returns_lag_13, returns_lag_14, returns_lag_15, returns_lag_16, returns_lag_17, returns_lag_18, returns_lag_19, returns_lag_20, returns_lag_21, returns_lag_22, returns_lag_23, returns_lag_24, returns_lag_25, returns_lag_26, returns_lag_27, returns_lag_28, returns_lag_29, returns_lag_30, returns_lag_31, returns_lag_32, returns_lag_33, returns_lag_34, returns_lag_35, returns_lag_36, returns_lag_37

In [ ]:
importlib.reload(dnn)

In [11]:
dnn.loop_classifier(
    hu=2000,
    fm=fm,
    laps=5,
    batch_size=20,
    set_class_weight=False,
    save_check_point=False,
    early_stopping=True,
    gpu=False,
    write_to_file=True)

Shuffling the data...
Splitting the data...
Data preparation completed.
Epoch 1/200


2023-01-21 02:28:41.270021: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2376000000 exceeds 10% of free system memory.


61/61 - 8s - loss: 1.5353 - tp: 583.0000 - fp: 544.0000 - tn: 1868.0000 - fn: 623.0000 - accuracy: 0.6774 - precision: 0.5173 - recall: 0.4834 - auc: 0.6940 - prc: 0.5332 - val_loss: 1.1935 - val_tp: 145.0000 - val_fp: 92.0000 - val_tn: 424.0000 - val_fn: 113.0000 - val_accuracy: 0.7351 - val_precision: 0.6118 - val_recall: 0.5620 - val_auc: 0.7667 - val_prc: 0.6287 - 8s/epoch - 128ms/step
Epoch 2/200


2023-01-21 02:28:46.742934: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2376000000 exceeds 10% of free system memory.


61/61 - 5s - loss: 0.6636 - tp: 903.0000 - fp: 264.0000 - tn: 2148.0000 - fn: 303.0000 - accuracy: 0.8433 - precision: 0.7738 - recall: 0.7488 - auc: 0.9034 - prc: 0.8312 - val_loss: 1.0918 - val_tp: 168.0000 - val_fp: 78.0000 - val_tn: 438.0000 - val_fn: 90.0000 - val_accuracy: 0.7829 - val_precision: 0.6829 - val_recall: 0.6512 - val_auc: 0.8168 - val_prc: 0.6919 - 5s/epoch - 90ms/step
Epoch 3/200


...

61/61 - 6s - loss: 0.4475 - tp: 1007.0000 - fp: 167.0000 - tn: 2245.0000 - fn: 199.0000 - accuracy: 0.8988 - precision: 0.8578 - recall: 0.8350 - auc: 0.9507 - prc: 0.9154 - val_loss: 0.9998 - val_tp: 165.0000 - val_fp: 85.0000 - val_tn: 431.0000 - val_fn: 93.0000 - val_accuracy: 0.7700 - val_precision: 0.6600 - val_recall: 0.6395 - val_auc: 0.8291 - val_prc: 0.7065 - 6s/epoch - 91ms/step
Epoch 4/200


2023-01-21 02:28:57.756578: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2376000000 exceeds 10% of free system memory.


61/61 - 5s - loss: 0.3470 - tp: 1050.0000 - fp: 139.0000 - tn: 2273.0000 - fn: 156.0000 - accuracy: 0.9185 - precision: 0.8831 - recall: 0.8706 - auc: 0.9675 - prc: 0.9416 - val_loss: 1.4800 - val_tp: 159.0000 - val_fp: 95.0000 - val_tn: 421.0000 - val_fn: 99.0000 - val_accuracy: 0.7494 - val_precision: 0.6260 - val_recall: 0.6163 - val_auc: 0.7820 - val_prc: 0.6469 - 5s/epoch - 89ms/step
Epoch 5/200


2023-01-21 02:29:03.359693: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2376000000 exceeds 10% of free system memory.


61/61 - 6s - loss: 0.2676 - tp: 1082.0000 - fp: 113.0000 - tn: 2299.0000 - fn: 124.0000 - accuracy: 0.9345 - precision: 0.9054 - recall: 0.8972 - auc: 0.9791 - prc: 0.9618 - val_loss: 1.0712 - val_tp: 179.0000 - val_fp: 72.0000 - val_tn: 444.0000 - val_fn: 79.0000 - val_accuracy: 0.8049 - val_precision: 0.7131 - val_recall: 0.6938 - val_auc: 0.8435 - val_prc: 0.7305 - 6s/epoch - 92ms/step
Epoch 6/200
61/61 - 6s - loss: 0.1512 - tp: 1138.0000 - fp: 55.0000 - tn: 2357.0000 - fn: 68.0000 - accuracy: 0.9660 - precision: 0.9539 - recall: 0.9436 - auc: 0.9938 - prc: 0.9887 - val_loss: 0.9541 - val_tp: 182.0000 - val_fp: 69.0000 - val_tn: 447.0000 - val_fn: 76.0000 - val_accuracy: 0.8127 - val_precision: 0.7251 - val_recall: 0.7054 - val_auc: 0.8664 - val_prc: 0.7701 - 6s/epoch - 91ms/step
Epoch 7/200
61/61 - 5s - loss: 0.1270 - tp: 1157.0000 - fp: 42.0000 - tn: 2370.0000 - fn: 49.0000 - accuracy: 0.9748 - precision: 0.9650 - recall: 0.9594 - auc: 0.9949 - prc: 0.9910 - val_loss: 0.9928 - val

In [ ]:
classifier = dnn.MultiDNNClassifer()

classifier.prepare_data(
    data = fm.df,
    cols = fm.cols,
    is_shuffle = True,
    y_to_categorical=True,
    rebalance = None,
    random_state=100,
    target_col="trade_signal"
)

initial_bias = utils.init_imbalanced_bias(
    data=classifier.data_train,
    target_col="trade_signal"
)

classifier.configure(
    hu = 100, 
    dropout=True, 
    input_dim=len(fm.cols),
    output_bias=initial_bias,
    class_num=3
)